In [2]:
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import sklearn
from sklearn.datasets import make_classification
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score,roc_auc_score

In [5]:
import pandas as pd
import gc
import numpy as np
table_1 = pd.read_table('open_data_train_valid/train/train_1.txt',sep = '\t')
columns = table_1.columns
table_2 = pd.read_table('open_data_train_valid/train/train_2.txt',sep = '\t',names= columns)
data = pd.concat([table_1,table_2],axis=0,ignore_index = True)
label = data['label']
label = label.iloc[label[label.notnull()].index.values]
data = data.iloc[label[label.notnull()].index.values]
del table_1
del table_2
gc.collect()

128

In [6]:
numerical_feature = np.load('open_data_train_valid/float64.npy')
factor_feature   = np.load('open_data_train_valid/int64.npy')
numerical = data[numerical_feature]
factor = data[factor_feature]
numerical.fillna(numerical.mean(axis=0),axis=0,inplace = True)
data = pd.concat([numerical,factor],axis=1)

//anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [86]:
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)
X_train,X_test,y_train,y_test = train_test_split(data,label,test_size = 0.2,random_state=0)

In [112]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_0,y_0 = X_train[y_train==0],y_train[y_train==0]
X_1,y_1 = X_train[y_train==1],y_train[y_train==1]
num = X_1.shape[0]

In [160]:
params_good = {
    'booster' : 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 2, #minimum loss reduction required to make a further partition on a leaf node of the tree
    'max_depth': 4,
    'alpha': 4,
    'subsample' : 0.8, #proportion of train set used to train
    'colsample_tree' : 0.8, #proportion of sample of features
    'min_child_weight': 0.8,
    'silent': 0, #print process
    'eta' : 0.1, # step size
}
plst_good = params_good.items()
model = xgb.XGBModel(plst_good)

In [165]:
num_train = 5
kf = KFold(n_splits=num_train)
count = 0
for test_index,train_index in kf.split(X_0):
    X,y = X_0[train_index],y_0[train_index]
    X_t ,y_t  = X_0[test_index],y_0[test_index]
    X = np.concatenate([X,X_1],axis=0)
    X_t = np.concatenate([X_t,X_1],axis=0)
    y = np.concatenate([y,y_1])
    y_t = np.concatenate([y_t,y_1],axis=0)
    dtrain = xgb.DMatrix(X,y)
    num_rounds = 25
    if count == 0:
        model = xgb.train(plst_good,dtrain,num_boost_round=num_rounds)
        count += 1
    else:
        print(1)
        model = xgb.train(plst_good,dtrain,num_boost_round=num_rounds,xgb_model=model)

[08:38:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[08:38:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[08:38:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:38:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:38:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:38:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[08:38:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:38:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[08:38:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_

[08:39:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 4 pruned nodes, max_depth=4
1
[08:39:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[08:39:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[08:39:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[08:39:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:39:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=4
[08:39:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:39:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[08:39:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, ma

In [166]:
y_pred = model.predict(xgb.DMatrix(X))
auc1 = roc_auc_score(y,y_pred)

In [175]:
auc1

0.9499376071568215

In [167]:
auc = roc_auc_score(y_test,model.predict(xgb.DMatrix(X_test)))

In [168]:
auc

0.8242846189446135

In [169]:
y_pred_test = model.predict(xgb.DMatrix(X_test))
y_pred_test[y_pred_test>0.5] = 1
y_pred_test[y_pred_test<0.5] = 0
accuracy_score(y_test,y_pred_test)
print(Counter(y_test[y_test!=y_pred_test]))
print(Counter(y_test))

Counter({0.0: 341, 1.0: 275})
Counter({0.0: 6252, 1.0: 441})


In [170]:
y_pred_train = model.predict(xgb.DMatrix(X_train))
y_pred_train[y_pred_train>0.5] = 1
y_pred_train[y_pred_train<0.5] = 0
accuracy_score(y_train,y_pred_train)
print(Counter(y_train[y_train!=y_pred_train]))
print(Counter(y_train))

Counter({0.0: 1048, 1.0: 606})
Counter({0.0: 25015, 1.0: 1757})


In [173]:
valid_set = pd.read_csv('open_data_train_valid/valid.txt',sep='\t')
numerical_data = valid_set[numerical_feature]
factor_data = valid_set[factor_feature]
numerical_data.fillna(numerical_data.mean(axis=0),axis=0,inplace = True)
#numerical_feat = scaler.transform(numerical_data)
#numerical_feat = pd.DataFrame(numerical_feat,index = numerical_data.index,columns=numerical_data.columns)
valid_data = pd.concat([numerical_data,factor_data],axis=1)
valid_data = scaler.transform(valid_data)
dvalid = xgb.DMatrix(valid_data)
y_valid_pre = model.predict(dvalid)


//anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [174]:
def write_file(filename,proba):
    with open(filename, 'w') as f:
        f.write('id,prob')
        f.write('\n')
        for i in range(proba.shape[0]):
            line = str(100000+i+1) + ',' + str(proba[i])
            f.write(line + '\n')
write_file('valid_sis.txt',y_valid_pre)